# ETL International airlines data
With this ETL Process we all going to get data about airlines data from International Airlines - Airline by country of port data. This data contains airlines names, number of passanger for each year from 1985, ports country of each airlines, number of passengers in, number of passengers out, etc.


# Extract (E) Step
In this step we are going to gather the dara from a csv file. This file we are going to get it from this url https://data.gov.au/data/dataset/ad89b4ff-541a-4729-b93c-4d2f5682e4c8/resource/809c77d8-fd68-4a2c-806f-c63d64e69842/download/airline_portcountry.csv

In [9]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [10]:
URL = "https://data.gov.au/data/dataset/ad89b4ff-541a-4729-b93c-4d2f5682e4c8/resource/809c77d8-fd68-4a2c-806f-c63d64e69842/download/airline_portcountry.csv"

# Limit quantity of records because free heroku tier allows 10,000.
# and left free space for anothers datasets examples
dataframe = pd.read_csv(URL, nrows=5000)

In [11]:
dataframe.head()

,Month,Airline,Port_Country,Passengers_In,Freight_In_(tonnes),Mail_In_(tonnes),Passengers_Out,Freight_Out_(tonnes),Mail_Out_(tonnes),Year,Month_num
0,Jan-85,Air Caledonie,New Caledonia,725,0.368,0.0,474,0.167,0.0,1985,1
1,Jan-85,Air China,China,1311,9.158,0.0,1232,2.587,0.0,1985,1
2,Jan-85,Air India,India,1423,44.823,0.0,970,6.708,0.0,1985,1
3,Jan-85,Air India,Singapore,501,10.583,0.0,306,124.966,0.0,1985,1
4,Jan-85,Air Nauru,Nauru,161,0.212,0.0,96,23.996,0.0,1985,1


# Transform (T) Step
In this step we are going to remove some columns that we do not need for this ETL Process or analysis. Also we are going to make the columns names lowercases

In [12]:
columns_to_remove = ['Month', 'Freight_In_(tonnes)', 'Mail_In_(tonnes)', 'Freight_Out_(tonnes)',
'Mail_Out_(tonnes)']

transformed_df = dataframe.drop(columns_to_remove, axis=1)
transformed_df.columns = map(str.lower, transformed_df.columns)

In [13]:
transformed_df.head()

,airline,port_country,passengers_in,passengers_out,year,month_num
0,Air Caledonie,New Caledonia,725,474,1985,1
1,Air China,China,1311,1232,1985,1
2,Air India,India,1423,970,1985,1
3,Air India,Singapore,501,306,1985,1
4,Air Nauru,Nauru,161,96,1985,1


# Load (L) Step
In this step we are going to load the data to PostgreSQL hosted on heroku

In [14]:
# Load enviroment variables
load_dotenv()

True

In [17]:
DB_URL = os.environ.get('DATABASE_URL')
engine = create_engine(DB_URL, echo=False)

In [18]:
transformed_df.to_sql('airlines', con=engine, if_exists="replace")

# Litter Example of a Analysis
We are going to look for the total of passangers that traveled using the airline "Japan Airlines" in 1985.

In [29]:
sql = """
    SELECT passengers_in, passengers_out from airlines 
    WHERE airline ='Japan Airlines' AND year = 1985
"""
results = engine.execute(sql).fetchall()

total_passengers = 0

# Number of in and out are inside a tuple, it sum between them and then sum that total to the total of passenger in 1985
for n_passengers in results:
    total_passengers += sum(n_passengers)

print("In year 1985, ", f'{total_passengers:,}', " passengers used 'Japan Airlines'")

In year 1985,  99,993  passengers used 'Japan Airlines'
